In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-analyst-job-postings-google-search/gsearch_jobs.csv


In [2]:
!pip install PyMySql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-analyst-job-postings-google-search/gsearch_jobs.csv


In [4]:
import gc 
gc.collect()

69

In [5]:
import pandas as pd

# Load the CSV file into a Pandas dataframe
jobposting_df = pd.read_csv("/kaggle/input/data-analyst-job-postings-google-search/gsearch_jobs.csv")

# Display the first 5 rows of the dataframe
print(jobposting_df.head())

   Unnamed: 0  index                    title      company_name  \
0           0      0    Business Data Analyst      Aurora Solar   
1           1      1      Senior Data Analyst     Tendo Systems   
2           2      2  Data Analyst III-Remote         Corporate   
3           3      3          Data Analyst II  Summit Utilities   
4           4      4  Data Analyst III-Remote         Corporate   

                  location                              via  \
0         United States                     via Ai-Jobs.net   
1                Anywhere                  via Startup Jobs   
2          Columbia, MO     via Central Illinois Proud Jobs   
3        Fort Smith, AR                 via Relocation Jobs   
4    Jefferson City, MO     via Central Illinois Proud Jobs   

                                         description  \
0  About the role:Aurora Solar is a fast-growing ...   
1  As a Senior Data Analyst, you will play a cruc...   
2  You could be the one who changes everything fo.

In [6]:
# Info about job posting table
jobposting_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15964 entries, 0 to 15963
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           15964 non-null  int64  
 1   index                15964 non-null  int64  
 2   title                15964 non-null  object 
 3   company_name         15964 non-null  object 
 4   location             15950 non-null  object 
 5   via                  15964 non-null  object 
 6   description          15964 non-null  object 
 7   extensions           15964 non-null  object 
 8   job_id               15964 non-null  object 
 9   thumbnail            8565 non-null   object 
 10  posted_at            15964 non-null  object 
 11  schedule_type        15862 non-null  object 
 12  work_from_home       7229 non-null   object 
 13  salary               3057 non-null   object 
 14  search_term          15964 non-null  object 
 15  date_time            15964 non-null 

In [7]:
from sqlalchemy import create_engine

!pip install PyMySql

import pymysql

import sqlalchemy

import sqlite3

In [8]:
# creating an engine

engine = create_engine('sqlite:////kaggle/working/Prabs1.db')

In [9]:
#Using the engine, we are porting jobposting table by writing records stored in DataFrames
jobposting_df.to_sql(name = "Job_Posting", con = engine, if_exists='replace', index=True, chunksize=10000)

In [10]:
%sql sqlite:////kaggle/working/Prabs.db

In [11]:
import pymysql

import sqlalchemy

import sqlite3

In [12]:
%sql sqlite:////kaggle/working/Prabs1.db

1. How many job postings require a Master's degree or higher?

In [13]:
%%sql
SELECT COUNT(*) AS master_required_count
FROM Job_Posting
WHERE description LIKE '%master%' OR description LIKE '%phd%' OR description LIKE '%doctoral%'


   sqlite:////kaggle/working/Prabs.db
 * sqlite:////kaggle/working/Prabs1.db
Done.


master_required_count
3157


2. How many job postings are work-from-home and what is the average salary range for these postings?

In [23]:
%%sql
SELECT 
    (SELECT COUNT(*) 
     FROM Job_Posting 
     WHERE work_from_home = '1') AS work_from_home_count, 
    round(AVG(salary_avg), 2) AS avg_salary
FROM Job_Posting 
WHERE work_from_home = '1'



   sqlite:////kaggle/working/Prabs.db
 * sqlite:////kaggle/working/Prabs1.db
Done.


work_from_home_count,avg_salary
7229,22389.55


3. What is the average salary range for data analyst job postings in the top 5 locations with the highest number of job postings?

In [22]:
%%sql
WITH location_cte AS (
    SELECT location, COUNT(*) AS posting_count
    FROM Job_Posting
    GROUP BY location
    ORDER BY posting_count DESC
    LIMIT 5
)
SELECT 
    location_cte.location, 
    round(AVG(salary_avg),2) AS avg_salary
FROM Job_Posting
JOIN location_cte ON Job_Posting.location = location_cte.location
WHERE salary_avg IS NOT NULL
GROUP BY location_cte.location


   sqlite:////kaggle/working/Prabs.db
 * sqlite:////kaggle/working/Prabs1.db
Done.


location,avg_salary
Anywhere,22288.62
"Jefferson City, MO",89310.77
"Kansas City, MO",57692.56
"Oklahoma City, OK",76828.01
United States,64285.83


4. How many job postings have a salary range above the national average for data analyst positions?

In [24]:
%%sql
WITH national_avg_cte AS (
    SELECT AVG(salary_avg) AS national_avg
    FROM Job_Posting
    WHERE salary_avg IS NOT NULL
)
SELECT COUNT(*) AS high_salary_count
FROM Job_Posting
WHERE salary_avg > (
    SELECT national_avg FROM national_avg_cte
)



   sqlite:////kaggle/working/Prabs.db
 * sqlite:////kaggle/working/Prabs1.db
Done.


high_salary_count
1308


5. How many job postings require SQL or Python, and what is the average salary range for each skill?

In [21]:
%%sql
WITH skill_counts AS (
  SELECT 
    SUM(CASE WHEN LOWER(description) LIKE '%sql%' THEN 1 ELSE 0 END) AS sql_count,
    SUM(CASE WHEN LOWER(description) LIKE '%python%' THEN 1 ELSE 0 END) AS python_count
  FROM Job_Posting
),
sql_stats AS (
  SELECT 
    COUNT(*) AS job_count,
    ROUND(AVG(salary_standardized),2) AS avg_salary,
    MIN(salary_standardized) AS min_salary,
    MAX(salary_standardized) AS max_salary
  FROM Job_Posting
  WHERE LOWER(description) LIKE '%sql%' AND salary_standardized IS NOT NULL
),
python_stats AS (
  SELECT 
    COUNT(*) AS job_count,
    ROUND(AVG(salary_standardized),2) AS avg_salary,
    MIN(salary_standardized) AS min_salary,
    MAX(salary_standardized) AS max_salary
  FROM Job_Posting
  WHERE LOWER(description) LIKE '%python%' AND salary_standardized IS NOT NULL
)
SELECT 
  skill_counts.sql_count, 
  sql_stats.job_count AS sql_job_count, 
  sql_stats.avg_salary AS sql_avg_salary, 
  sql_stats.min_salary AS sql_min_salary, 
  sql_stats.max_salary AS sql_max_salary,
  skill_counts.python_count, 
  python_stats.job_count AS python_job_count, 
  python_stats.avg_salary AS python_avg_salary, 
  python_stats.min_salary AS python_min_salary, 
  python_stats.max_salary AS python_max_salary
FROM skill_counts
CROSS JOIN sql_stats
CROSS JOIN python_stats;



   sqlite:////kaggle/working/Prabs.db
 * sqlite:////kaggle/working/Prabs1.db
Done.


sql_count,sql_job_count,sql_avg_salary,sql_min_salary,sql_max_salary,python_count,python_job_count,python_avg_salary,python_min_salary,python_max_salary
8832,1491,101117.97,18720.0,260000.0,4668,632,103242.08,18720.0,322400.0


6. How many job postings have a salary above $100,000, and what is the average salary for those postings?

In [36]:
%%sql
WITH high_salary_jobs AS (
  SELECT * 
  FROM Job_Posting
  WHERE salary_standardized IS NOT NULL AND salary_standardized > 100000
    AND LOWER(title) LIKE '%analyst%'
)
SELECT COUNT(*) AS job_count, ROUND(AVG(salary_standardized),2) AS avg_salary
FROM high_salary_jobs;


   sqlite:////kaggle/working/Prabs.db
 * sqlite:////kaggle/working/Prabs1.db
Done.


job_count,avg_salary
830,129899.33
